In [1]:
#regualr spacy instal lwould not work, had to get from github
#conda install -c conda-forge spacy
#python -m spacy download en
#check with: python -m spacy validate
#restart jupyter notebook

import codecs

import spacy
import pandas as pd
import itertools as it
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

import en_core_web_sm
#nlp = en_core_web_sm.load()
#import en_core_web_md
#nlp = en_core_web_md.load()
nlp = spacy.load('en')

In [2]:
#test
with codecs.open('review_text_lv_rest_subset_30.txt', encoding='utf_8') as f:
    sample_review = list(it.islice(f, 8, 9))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

I live very near this place and have been curious to try it for sometime now as it always seems busy.

Well my curiosity has been satisfied and I'm sure I'll never feel the need to visit this place again.

It struck me as a typical trendy place that yuppies feel they need to visit as all the other yuppies do.  
"Oh the food is sooo good!!"
"Really? You've eaten there?"
"No. But all my yuppy friends have and they say it's good so it must be."

I only give it a two because the service and presentation of the food was decent.

We ordered something called oxtail fries which were horrible. I'm sure an actual oxtail would probably have been tastier.

My girlfriend and I each ordered a different burger so we could sample each others. Sorry I can't remember what they were called but we didn't care for either one. The best compliment I could pay them is to say they were at least edible.

So as mentioned the service was decent and the food looked presentable but as far as taste I can't recommend

In [3]:
#helper functions from modern nlp in python
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [4]:
unigram_sentences_filepath = 'unigram_sentences_all_eat_30.txt'

In [6]:
%%time
if 1 == 0:
    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus('review_text_eats_subset_30.txt'):
            f.write(sentence + '\n')

CPU times: user 2d 14h 54min 22s, sys: 1d 16h 18min 7s, total: 4d 7h 12min 29s
Wall time: 15h 9min 21s


In [7]:
%%time
if 1 == 0:
    with codecs.open('unigram_sentences_all_lv_rest_30.txt', 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus('review_text_lv_rest_subset_30.txt'):
            f.write(sentence + '\n')

CPU times: user 17h 53min 55s, sys: 12h 6min 47s, total: 1d 6h 43s
Wall time: 4h 29min 35s


In [5]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

#build models with larger dataset

In [6]:
for unigram_sentence in it.islice(unigram_sentences, 230, 240):
    print( u' '.join(unigram_sentence))
    print(u'')

at first -PRON- be not impress that -PRON- be

a 45 min wait however -PRON- bf really want to try this piece so -PRON- decide to wait

-PRON- be definitely different and worth the wait

-PRON- start off with the house cured meat -PRON- be good but definitely too much for just 2 people to share and no need to order the extra bread if -PRON- be only two people

next come the roasted bone marrow which be amazing and definitely worth ordering

have -PRON- not order so many other dish -PRON- would have order a second round of this

who know bone marrow could be this tasty

then come the spicy horse tartare which -PRON- think be intriguing as -PRON- have never hear so -PRON- have order -PRON-

when -PRON- show up -PRON- be like a salad

be not bad worth try but probably only a one time thing for -PRON-



In [9]:
bigram_model_filepath = 'bigram_model_all_eat_30'


In [11]:
%%time

#time consuming
if 1 == 1:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

CPU times: user 8min 49s, sys: 5.43 s, total: 8min 54s
Wall time: 8min 54s


In [12]:
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

In [13]:
bigram_sentences_filepath = 'bigram_sentences_all_eat_30.txt'

In [14]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 19min 41s, sys: 5.7 s, total: 19min 47s
Wall time: 19min 50s


In [15]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [16]:
for bigram_sentence in it.islice(bigram_sentences, 230, 270):
    print(u' '.join(bigram_sentence))
    print(u'')    

at first -PRON- be not impress that -PRON- be

a 45_min wait however -PRON- bf really want to try this piece so -PRON- decide to wait

-PRON- be definitely different and worth the wait

-PRON- start off with the house_cured meat -PRON- be good but definitely too much for just 2 people to share and no need to order the extra bread if -PRON- be only two people

next come the roasted bone_marrow which be amazing and definitely worth ordering

have -PRON- not order so many other dish -PRON- would have order a second round of this

who know bone_marrow could be this tasty

then come the spicy horse_tartare which -PRON- think be intriguing as -PRON- have never hear so -PRON- have order -PRON-

when -PRON- show up -PRON- be like a salad

be not bad worth try but probably only a one time thing for -PRON-

tongue on brioche be definitely another worth order

-PRON- be a sandwich that taste like a montreal smoke meat sandwich

so good that -PRON- melt in -PRON- mouth so soft and tender

if -PRON

In [17]:
trigram_model_filepath = 'trigram_model_all_eat_30'

In [18]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 1:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)

CPU times: user 7min 55s, sys: 2.22 s, total: 7min 58s
Wall time: 7min 55s


In [19]:
trigram_model = Phrases.load(trigram_model_filepath)

In [20]:
trigram_sentences_filepath = 'trigram_sentences_all_eat_30.txt'

In [21]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 19min 36s, sys: 5.76 s, total: 19min 41s
Wall time: 19min 45s


In [22]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [27]:
for trigram_sentence in it.islice(trigram_sentences, 320, 520):
    print(u' '.join(trigram_sentence))
    print(u'')

stumble_upon this one day thought -PRON- would give -PRON- a try

typical sushi place sushi be pretty good

price be decent

party tray be decent

=)

fish be good really good

love how -PRON- could have so many different option for fish choice

chip be only ok

onion_ring be not the same as the regular onion a little too crispy

-PRON- have order the house organic batter mahi_mahi very light not really heavy

creamy coleslaw be flavourless

share the o.t.h poutine which be ok but not for -PRON-

will go back to try the pull_pork or duck_confit one

good for gluten_free

=)

after read the review on yelp -PRON- boyfriend and i decide to give this place a try especially since -PRON- be on a mission to try raman in every city -PRON- visit

-PRON- be really excited as when -PRON- be in nyc -PRON- have really good raman

listen to the review -PRON- start off with the gyoza appetizer which be not as good as everyone say -PRON- be

-PRON- be pan_fry with the thin_layer of skin but there be a

In [24]:
#Write the transformed text out to a new file, with one review per line.

trigram_reviews_filepath = 'trigram_transformed_reviews_lv_rest.txt'



In [26]:
review_txt_filepath = 'review_text_lv_rest_subset_30.txt'

In [ ]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.lang.en.stop_words.STOP_WORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [29]:
#spacy.lang.en.stop_words.STOP_WORDS

{'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'front',
 'full',
 'further',
 'get',
 'give',
 'g